In [ ]:
#camel case and snake case


In [ ]:
             "poster": poster,
                 "movie_page": movie_page}

    return movie_info


def select_movie_number(movie_info_list):
    for i in range(len(movie_info_list)):
        single_movie_info = movie_info_list[i]
        print(f'{i+1} : {single_movie_info["title"]} - {single_movie_info["year"]}')

    selected_movie = int(input("Enter the movie number: "))
    return movie_info_list[selected_movie - 1]
              

def subtitle_html_to_dict(single_subtitle):
    subtitle_language = single_subtitle.find(".sub-lang", first=True).text

    download_cell = single_subtitle.find('.download-cell', first=True)
    download_page_link = download_cell.find("a", first=True).absolute_links.pop()

    subtitle_rating = int(single_subtitle.find(".rating-cell", first=True).text)

    subtitle_dict = {"language": subtitle_language,
                    "download_page": download_page_link,
                    "rating": subtitle_rating}

    return subtitle_dict


def filter_subtitles(subtitle_info_list, filter_language="English"):
    results = []
    for single_subtitle in subtitle_info_list:
        if single_subtitle['language'] == filter_language:
            results.append(single_subtitle)

    if len(results) == 0:
        selected_subtitle = None
    else:
        selected_subtitle = results[0]

    return selected_subtitle


def download_page_to_zip_url(final_subtitle):
    response = url_to_response(final_subtitle["download_page"])
    download_link = response.html.find('.download-subtitle', first=True).attrs["href"]
    return download_link


def download_zip_file(zip_url):
    zip_file_data = requests.get(zip_url).content
    filename = zip_url.split("/")[-1]

    with open(filename, "wb") as zip_file:
        zip_file.write(zip_file_data)
    
    return 0


def main():
    query = input("Enter movie name: ") # take movie name from user
    response = search_for_subtitle(query) # search for movie on website
    search_results = response.html.find('.media-movie-clickable') # get all the results

    if len(search_results) > 0:
        print(f"found {len(search_results)} subtitles. ")
    else:
        print("No results found.")
        return 0 # exit is no result is found

    # take search results convert them into a dict
    movie_info_list = list(map(search_result_to_movie_info, search_results))

    # Ask user to select a movie from the list
    selected_movie = select_movie_number(movie_info_list)

    # get the selected movie page
    movie_page_response = url_to_response(selected_movie["movie_page"])

    # find the list of subtitles for that movie
    subtitle_list = movie_page_response.html.find('.high-rating')
 
    # convert html subtitles to dict
    subtitle_info_list = list(map(subtitle_html_to_dict, subtitle_list))
    
    # select subtitle based on language and rating
    final_subtitle = filter_subtitles(subtitle_info_list)
    
    # get the zip file url from the movie page
    zip_url = download_page_to_zip_url(final_subtitle)
    
    # download the zip file
    download_zip_file(zip_url)
    print("Downloaded Subtitle in the current Directory.")
          

main()